In [90]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [91]:
# Import indicator dataframe
# Start with 1-minute... alghough we would like to attatch them all together soon.
df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader/resources/aapl_3min_pivot_point_indicator_df.csv")
# df.head()

# df = pd.read_csv("../algotrader/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()



,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
0,2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,...,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0,1
1,2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,...,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0,1
2,2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,...,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0,1
3,2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,...,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0,1
4,2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,...,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0,1


In [92]:
# our df does not have date-time
hello = df['timestamp']
hello

0         2023-01-03 15:04:00+00:00
1         2023-01-03 15:05:00+00:00
2         2023-01-03 15:06:00+00:00
3         2023-01-03 15:07:00+00:00
4         2023-01-03 15:08:00+00:00
                    ...            
153677    2024-01-17 00:47:00+00:00
153678    2024-01-17 00:52:00+00:00
153679    2024-01-17 00:55:00+00:00
153680    2024-01-17 00:57:00+00:00
153681    2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: object

In [93]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])

hello = df['timestamp']
hello

0        2023-01-03 15:04:00+00:00
1        2023-01-03 15:05:00+00:00
2        2023-01-03 15:06:00+00:00
3        2023-01-03 15:07:00+00:00
4        2023-01-03 15:08:00+00:00
                    ...           
153677   2024-01-17 00:47:00+00:00
153678   2024-01-17 00:52:00+00:00
153679   2024-01-17 00:55:00+00:00
153680   2024-01-17 00:57:00+00:00
153681   2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: datetime64[ns, UTC]

In [94]:
df.set_index('timestamp', inplace=True)

df.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0,1
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0,1
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0,1
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0,1
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0,1


In [95]:
# X is everything except the new_signal column
X = df.drop('new_signal', axis=1)

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [96]:
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:05:00+00:00,126.5001,126.880,126.4166,5733.0,126.820,513722.0,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0.0
2023-01-03 15:06:00+00:00,126.8500,126.895,126.4400,3763.0,126.505,362900.0,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0.0
2023-01-03 15:07:00+00:00,126.5800,126.870,126.5300,2930.0,126.860,240388.0,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0.0
2023-01-03 15:08:00+00:00,126.7100,126.870,126.5000,3189.0,126.580,289721.0,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0.0
2023-01-03 15:09:00+00:00,126.7066,126.760,126.5601,2861.0,126.705,255144.0,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0.0


We may look to consider what it would do if we changed the amount that we shifted by. Perhaps we tried predicting 5 minutes into the future... how about an hour?

In [97]:
# y is the NEW signal column
y = df[("new_signal")]

y


timestamp
2023-01-03 15:04:00+00:00    1
2023-01-03 15:05:00+00:00    1
2023-01-03 15:06:00+00:00    1
2023-01-03 15:07:00+00:00    1
2023-01-03 15:08:00+00:00    1
                            ..
2024-01-17 00:47:00+00:00    1
2024-01-17 00:52:00+00:00    1
2024-01-17 00:55:00+00:00    1
2024-01-17 00:57:00+00:00    1
2024-01-17 00:58:00+00:00    1
Name: new_signal, Length: 153682, dtype: int64

In [98]:
# Set start of training period
training_begin = X.index.min()

print(training_begin)

2023-01-03 15:05:00+00:00


In [99]:
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)

print(training_end)

2023-10-03 15:05:00+00:00


In [100]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [101]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [102]:
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(113680, 98)

(40002, 98)

In [103]:
# from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

In [104]:
# display(X_train_scaled.shape)
# display(X_test_scaled.shape)

### Now that we have scaled our data, we can build our neural network.

In [105]:
num_predictors = len(X.columns)

# We have 2 possible outcomes, and we are trying to predict the stock/indicators to be in position -1 or 1
num_classes = 1

num_predictors

98

In [106]:
nn_model = Sequential()

In [107]:
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))

In [108]:
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))

# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

In [109]:
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))

In [110]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                990       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1001 (3.91 KB)
Trainable params: 1001 (3.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100


910/910 [==============================] - 1s 682us/step - loss: -2.6758 - accuracy: 0.0350 - val_loss: -2.9714 - val_accuracy: 0.0925
Epoch 2/100
910/910 [==============================] - 1s 589us/step - loss: -17.9165 - accuracy: 0.0632 - val_loss: -13.3242 - val_accuracy: 0.0965
Epoch 3/100
910/910 [==============================] - 0s 543us/step - loss: -46.6189 - accuracy: 0.0691 - val_loss: -30.0524 - val_accuracy: 0.1117
Epoch 4/100
910/910 [==============================] - 0s 525us/step - loss: -86.3838 - accuracy: 0.0699 - val_loss: -50.7285 - val_accuracy: 0.1031
Epoch 5/100
910/910 [==============================] - 0s 499us/step - loss: -135.5154 - accuracy: 0.0698 - val_loss: -76.3880 - val_accuracy: 0.1070
Epoch 6/100
910/910 [==============================] - 0s 492us/step - loss: -193.5571 - accuracy: 0.0695 - val_loss: -105.7193 - val_accuracy: 0.1074
Epoch 7/100
910/910 [==============================] - 0s 496us/step - loss: -259.4991 - accuracy: 0.0709 - val_loss:

In [112]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1251/1251 - 0s - loss: -2.2239e+04 - accuracy: 0.0919 - 356ms/epoch - 285us/step
Loss: -22239.267578125, Accuracy: 0.09189540892839432


Now we backtest with the TEST portion of the data

In [113]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

3553/3553 [==============================] - 1s 258us/step


In [114]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

In [115]:
# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

In [116]:
# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

In [117]:
# # Now we can plot the accuracy for training and validation

# training_results = pd.DataFrame(index=range(1, num_epochs+1))
# training_results['Training'] = model_history['categorical_accuracy']
# training_results['Validation'] = model_history['val_categorical_accuracy']
# training_results.plot(title = 'Training and Validation Performance')

### Now using the 3 minute

In [146]:
# Import indicator dataframe
df = pd.read_csv("../algotrader2/resources/aapl_3min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
0,2023-01-04 00:12:00+00:00,124.95,124.96,124.95,92,124.96,3977,124.952526,-0.00024,124.974,...,-1.427075,1.439150,False,-5.103025e+07,0.0,5.639625,0.031817,99.378906,0,1
1,2023-01-04 00:15:00+00:00,124.96,124.96,124.95,83,124.95,2583,124.949560,0.00008,124.970,...,5.886957,1.047844,False,-5.102509e+07,0.0,5.663836,0.027480,98.040465,0,1
2,2023-01-04 00:18:00+00:00,124.97,124.97,124.96,56,124.96,765,124.962646,0.00008,124.968,...,12.868265,0.858268,False,-5.102356e+07,0.0,6.360573,0.024794,96.496636,0,1
3,2023-01-04 00:21:00+00:00,125.03,125.10,124.98,128,124.98,11318,125.033116,0.00048,124.978,...,41.699712,1.045054,False,-5.101413e+07,0.0,6.359419,0.026735,96.194995,0,1
4,2023-01-04 00:24:00+00:00,125.05,125.05,125.02,38,125.04,1088,125.043639,0.00016,124.992,...,48.064587,1.318566,False,-5.101340e+07,0.0,6.669126,0.032957,97.328048,0,1


In [147]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('new_signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("new_signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-04 00:15:00+00:00,124.95,124.96,124.95,92.0,124.96,3977.0,124.952526,-0.00024,124.974,124.964,...,0.070644,-1.427075,1.439150,False,-5.103025e+07,0.0,5.639625,0.031817,99.378906,0.0
2023-01-04 00:18:00+00:00,124.96,124.96,124.95,83.0,124.95,2583.0,124.949560,0.00008,124.970,124.965,...,0.059256,5.886957,1.047844,False,-5.102509e+07,0.0,5.663836,0.027480,98.040465,0.0
2023-01-04 00:21:00+00:00,124.97,124.97,124.96,56.0,124.96,765.0,124.962646,0.00008,124.968,124.967,...,0.057214,12.868265,0.858268,False,-5.102356e+07,0.0,6.360573,0.024794,96.496636,0.0
2023-01-04 00:24:00+00:00,125.03,125.10,124.98,128.0,124.98,11318.0,125.033116,0.00048,124.978,124.974,...,0.073006,41.699712,1.045054,False,-5.101413e+07,0.0,6.359419,0.026735,96.194995,0.0
2023-01-04 00:27:00+00:00,125.05,125.05,125.02,38.0,125.04,1088.0,125.043639,0.00016,124.992,124.984,...,0.083011,48.064587,1.318566,False,-5.101340e+07,0.0,6.669126,0.032957,97.328048,0.0


Start date: 2023-01-04 00:15:00+00:00
End date: 2023-10-04 00:15:00+00:00


In [148]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

# from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
# display(X_train_scaled.shape)
# display(X_test_scaled.shape)

(47724, 98)

(16982, 98)

In [149]:
# NEURAL NETWORK
num_predictors = len(X.columns)
num_classes = 1
nn_model = Sequential()
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))
# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))


In [150]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 10)                990       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1001 (3.91 KB)
Trainable params: 1001 (3.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [151]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100
382/382 [==============================] - 0s 700us/step - loss: -0.7306 - accuracy: 0.0370 - val_loss: -1.5416 - val_accuracy: 0.0779
Epoch 2/100
382/382 [==============================] - 0s 547us/step - loss: -3.8820 - accuracy: 0.0505 - val_loss: -5.3175 - val_accuracy: 0.0845
Epoch 3/100
382/382 [==============================] - 0s 518us/step - loss: -9.8679 - accuracy: 0.0603 - val_loss: -11.6910 - val_accuracy: 0.1052
Epoch 4/100
382/382 [==============================] - 0s 519us/step - loss: -18.8370 - accuracy: 0.0661 - val_loss: -20.3364 - val_accuracy: 0.1161
Epoch 5/100
382/382 [==============================] - 0s 515us/step - loss: -30.5920 - accuracy: 0.0680 - val_loss: -30.8880 - val_accuracy: 0.1078
Epoch 6/100
382/382 [==============================] - 0s 515us/step - loss: -44.8082 - accuracy: 0.0701 - val_loss: -43.2986 - val_accuracy: 0.1116
Epoch 7/100
382/382 [==============================] - 0s 517us/step - loss: -61.2891 - accuracy: 0.0703 - val_

In [124]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

531/531 - 0s - loss: 0.0000e+00 - accuracy: 0.1104 - 205ms/epoch - 386us/step
Loss: 0.0, Accuracy: 0.11041102558374405


In [125]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

1492/1492 [==============================] - 0s 278us/step


In [126]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

### Now 15 minute

In [153]:
# Import indicator dataframe
df = pd.read_csv("../algotrader2/resources/aapl_15min_pivot_point_indicator_df.csv")
df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
0,2023-01-09 21:30:00+00:00,130.0817,130.15,130.0400,314,130.09,18713,130.096350,0.000167,130.19634,...,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0,0,1
1,2023-01-09 21:45:00+00:00,130.1000,130.14,130.1000,182,130.11,4711,130.121440,0.000141,130.11634,...,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0,0,1
2,2023-01-09 22:00:00+00:00,130.0600,130.12,130.0300,128,130.12,6111,130.076383,-0.000307,130.09034,...,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0,0,1
3,2023-01-09 22:15:00+00:00,130.0700,130.09,130.0315,210,130.06,95004,130.135637,0.000077,130.07434,...,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0,0,1
4,2023-01-09 22:30:00+00:00,130.0600,130.08,130.0300,148,130.04,4724,130.045182,-0.000077,130.07434,...,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0,0,1


In [154]:
df.head(100)

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal,new_signal
0,2023-01-09 21:30:00+00:00,130.0817,130.15,130.0400,314,130.09,18713,130.096350,0.000167,130.19634,...,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.000000,0,1
1,2023-01-09 21:45:00+00:00,130.1000,130.14,130.1000,182,130.11,4711,130.121440,0.000141,130.11634,...,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.000000,0,1
2,2023-01-09 22:00:00+00:00,130.0600,130.12,130.0300,128,130.12,6111,130.076383,-0.000307,130.09034,...,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.000000,0,1
3,2023-01-09 22:15:00+00:00,130.0700,130.09,130.0315,210,130.06,95004,130.135637,0.000077,130.07434,...,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.000000,0,1
4,2023-01-09 22:30:00+00:00,130.0600,130.08,130.0300,148,130.04,4724,130.045182,-0.000077,130.07434,...,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.000000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-01-11 13:30:00+00:00,131.2900,131.31,130.8700,2008,131.06,165553,131.065014,0.001755,130.92000,...,30.086174,-0.105599,False,-7.340530e+07,0.000000,12.636269,0.205320,2.672706,0,-1
96,2023-01-11 13:45:00+00:00,131.5000,131.68,131.2000,2137,131.32,170573,131.416796,0.001600,131.08000,...,39.894154,0.509515,False,-7.327737e+07,0.000000,12.711564,0.232669,9.345933,1,1
97,2023-01-11 14:00:00+00:00,131.3400,131.50,131.2500,1497,131.50,110944,131.365684,-0.001217,131.20800,...,25.043798,0.982269,True,-7.341938e+07,0.000000,12.648408,0.243546,21.903318,0,1
98,2023-01-11 14:15:00+00:00,131.2200,131.59,131.0600,2316,131.36,181803,131.326000,-0.000914,131.28200,...,14.881195,1.277639,True,-7.351543e+07,0.000000,12.570650,0.245828,39.585004,0,1


In [155]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('new_signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("new_signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-09 21:45:00+00:00,130.0817,130.15,130.0400,314.0,130.09,18713.0,130.096350,0.000167,130.19634,130.50517,...,-2.688380,-53.399594,-2.609553,False,-6.975113e+07,-22.378567,9.776626,1.066249,0.0,0.0
2023-01-09 22:00:00+00:00,130.1000,130.14,130.1000,182.0,130.11,4711.0,130.121440,0.000141,130.11634,130.36567,...,-2.717707,-51.671029,-2.459362,False,-6.975348e+07,-23.502623,10.152024,1.046721,0.0,0.0
2023-01-09 22:15:00+00:00,130.0600,130.12,130.0300,128.0,130.12,6111.0,130.076383,-0.000307,130.09034,130.26267,...,-2.637492,-52.958418,-2.603288,False,-6.976163e+07,-24.565646,9.791596,1.011924,0.0,0.0
2023-01-09 22:30:00+00:00,130.0700,130.09,130.0315,210.0,130.06,95004.0,130.135637,0.000077,130.07434,130.20267,...,-2.482767,-51.834922,-2.888556,False,-6.972915e+07,-25.441689,10.514562,0.966759,0.0,0.0
2023-01-09 22:45:00+00:00,130.0600,130.08,130.0300,148.0,130.04,4724.0,130.045182,-0.000077,130.07434,130.17167,...,-2.311230,-52.224826,-3.240848,False,-6.972537e+07,-26.410294,11.958663,0.907764,0.0,0.0


Start date: 2023-01-09 21:45:00+00:00
End date: 2023-10-09 21:45:00+00:00


In [156]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

# # from imblearn.over_sampling import RandomOverSampler
# # Use RandomOverSampler to resample the datase using random_state=1
# ros = RandomOverSampler(random_state=1)
# X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
# display(X_train_scaled.shape)
# display(X_test_scaled.shape)

(11440, 98)

(4046, 98)

In [157]:
# NEURAL NETWORK
num_predictors = len(X.columns)
num_classes = 1
nn_model = Sequential()
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))
# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="sigmoid"))


In [158]:
# Compile model
nn_model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 10)                990       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 1001 (3.91 KB)
Trainable params: 1001 (3.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [159]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100
92/92 [==============================] - 0s 1ms/step - loss: 0.0269 - accuracy: 0.0343 - val_loss: -0.2735 - val_accuracy: 0.0603
Epoch 2/100
92/92 [==============================] - 0s 646us/step - loss: -0.4203 - accuracy: 0.0294 - val_loss: -0.7045 - val_accuracy: 0.0953
Epoch 3/100
92/92 [==============================] - 0s 641us/step - loss: -0.8997 - accuracy: 0.0367 - val_loss: -1.2922 - val_accuracy: 0.1149
Epoch 4/100
92/92 [==============================] - 0s 678us/step - loss: -1.5659 - accuracy: 0.0500 - val_loss: -2.0391 - val_accuracy: 0.1075
Epoch 5/100
92/92 [==============================] - 0s 670us/step - loss: -2.4415 - accuracy: 0.0527 - val_loss: -3.0577 - val_accuracy: 0.0992
Epoch 6/100
92/92 [==============================] - 0s 667us/step - loss: -3.5537 - accuracy: 0.0547 - val_loss: -4.3136 - val_accuracy: 0.0927
Epoch 7/100
92/92 [==============================] - 0s 657us/step - loss: -4.8842 - accuracy: 0.0578 - val_loss: -5.7085 - val_accur

In [134]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1251/1251 - 0s - loss: 0.0000e+00 - accuracy: 0.1130 - 364ms/epoch - 291us/step
Loss: 0.0, Accuracy: 0.11304435133934021


In [135]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

3553/3553 [==============================] - 1s 274us/step


In [136]:
nn_test_predictions[:500]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [137]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

In [138]:
# # Deployment (Hypothetical)
# while True:
#     current_data = your_trading_api.get_real_time_data()
#     current_data_processed = preprocess_data(current_data)
#     prediction = model.predict(current_data_processed)
#     if prediction > some_threshold:
#         your_trading_api.execute_trade()

# # Placeholder Functions
# def combine_data(historical, news):
#     # Combine and return data
#     pass

# def split_data(data):
#     # Split and return data
#     pass

# def backtest_strategy(model, data):
#     # Implement backtesting logic
#     pass

# def preprocess_data(data):
#     # Data preprocessing steps
#     pass